In [5]:
import os
import ocr
import fitz
import mimetypes
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np

In [16]:
def _pixmap_to_numpy(pixmap):
    im = np.frombuffer(pixmap.samples, dtype=np.uint8).reshape(pixmap.h, pixmap.w, pixmap.n)
    return np.ascontiguousarray(im[..., [2, 1, 0]])


def extract_page(fp: str, pn: int = None):
    if not os.path.exists(fp):
        raise "Provided file %s does not exist" % fp

    if not mimetypes.guess_type(fp)[0] == "application/pdf":
        raise "Provided file %s is not a valid PDF" % fp

    document = fitz.open(fp)
    if pn is None:
        for page in document:
            yield _pixmap_to_numpy(page.get_pixmap(matrix=fitz.Matrix(3, 3)))
    else:
        yield _pixmap_to_numpy(document.load_page(pn).get_pixmap(matrix=fitz.Matrix(3, 3)))


def show_image(image):
    cv.namedWindow("win", cv.WINDOW_FREERATIO)
    cv.imshow("win", image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
for page in extract_page(r"C:\Users\arish\OneDrive\Desktop\Monsanto.pdf"):
    # Write image to an output directory
    pass

In [ ]:
def crop_required_portions(img):
    g_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    thresh = cv.threshold(g_img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)[1]

    v_contours = _find_contours(thresh, (1, 10))
    h_contours = _find_contours(thresh, (10, 2))

    # v_lines = sorted(
    #     [_max(v_contours, 3), _max(v_contours, 3), _max(v_contours, 3)],
    #     key=lambda x: x[0][0],
    # )
    #
    # h_lines = sorted(
    #     [_max(h_contours, 2), _max(h_contours, 2), _max(h_contours, 2)],
    #     key=lambda x: x[0][1],
    # )
    #
    # for lines in v_lines + h_lines:
    #     cv2.drawContours(img, [lines[1]], 0, (255, 255, 255), 3)

    # return \
    #     img[h_lines[1][0][1]:h_lines[2][0][1], v_lines[0][0][0]:v_lines[2][0][0]], \
    #     img[h_lines[0][0][1]:h_lines[1][0][1], v_lines[1][0][0]:v_lines[2][0][0]]

# def _find_contours(_thresh, _size):
#     contours = cv.findContours(
#         cv.morphologyEx(
#             _thresh,
#             cv.MORPH_OPEN,
#             cv.getStructuringElement(cv.MORPH_RECT, _size),
#             iterations=2
#         ),
#         cv.RETR_EXTERNAL,
#         cv.CHAIN_APPROX_SIMPLE
#     )
#
#     contours = contours[0] if len(contours) == 2 else contours[1]
#
#     lines = []
#     for c in contours:
#         lines.append((cv2.boundingRect(c), c))
#
#     return lines

In [ ]:
image = list(extract_page(r"C:\Users\arish\OneDrive\Desktop\Monsanto.pdf", 0))[0]

In [21]:
# Greyscale Copy of image
g_img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

# Convert to Binary image, ignore the first value of tuple returned from
# cv.threshold since it represents the threshold number.
b_img = cv.threshold(g_img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)[1]

In [79]:
temp_img = image.copy()

# morph_kernel_shape = (10, 2)
morph_kernel_shape = (2, 10)

morph_kernel = cv.getStructuringElement(cv.MORPH_RECT, morph_kernel_shape)
morph_img = cv.morphologyEx(b_img, cv.MORPH_OPEN, morph_kernel, iterations=2)

# Save this image
# show_image(morph_img)

contours = cv.findContours(morph_img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
contours = list(contours[0] if len(contours) == 2 else contours[1])

cv.boundingRect(contours[0])
# show_image(temp_img)

(3500, 2298, 3, 31)

In [73]:
def horizontal_lines(contours):
    for c in contours:
        x, y, w, h = cv.boundingRect(c)

        yield c, (x, y, w, h), w


def vertical_lines(contours):
    bounds = []
    height = []

    for c in contours:
        x, y, w, h = cv.boundingRect(c)
        bounds.append((x, y, w, h))
        height.append(h)

    return contours, bounds, height

In [74]:
cnts, bounds, height = vertical_lines(contours)

# 3 Max values
first_index = np.argmax(height)
first = cnts[first_index]
del cnts[first_index]
del bounds[first_index]
del height[first_index]

second_index = np.argmax(height)
second = cnts[second_index]
del cnts[second_index]
del bounds[second_index]
del height[second_index]

third_index = np.argmax(height)
third = cnts[third_index]
del cnts[third_index]
del bounds[third_index]
del height[third_index]

In [77]:
cv.drawContours(temp_img, [first, second, third], -1, (0, 0, 255), 5)
show_image(temp_img)